# Synth1GAN
---
This code was shamelessly stolen from https://github.com/eriklindernoren/PyTorch-GAN
It uses a WGAN GAN architecture, which to be honest have no idea how it works but after trying WGANS and just Normal GANS these seem to work best

The model on www.thispatchdoesnotexist.com was trained on a GTX 1070 overnight (~8-10 hours). 

Model checkpoints will show up in ./checkpoints, and CSV outputs will appear in wgangp_final

To turn the CSV outputs back into Synth1 folders with presets, go back to the "feature_engineering" page to the section titled "Converting the GAN Output back to a Synth1 Bank"

In [ ]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

In [ ]:
import pandas

In [ ]:
class Ops:
    n_epochs = 20000
    batch_size = 64
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999
    n_cpu = 8
    latent_dim = 10
    img_size = 28
    channels = 1
    n_critic = 5
    clip_value = 0.01
    sample_interval = 400

In [ ]:
opt = Ops()

In [ ]:
img_shape = (opt.channels, opt.img_size, opt.img_size)


In [ ]:
from torch.utils.data import Dataset

class PresetDataset(Dataset):
    
    def __init__(self,filepath):
        self.data = pandas.read_csv(filepath)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return torch.FloatTensor(list(self.data.iloc[idx]))

In [ ]:
preset_dataset = PresetDataset("./presets_all_norm_minmax_one_hot_neg.csv")
data_size = preset_dataset[101].size()[0]

In [ ]:
data_size

In [ ]:
os.makedirs("output", exist_ok=True)

In [ ]:
cuda = True if torch.cuda.is_available() else False


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, data_size),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(data_size, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 1),
        )

    def forward(self, img):
        img_flat = img
        validity = self.model(img_flat)
        return validity

In [ ]:

# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()


In [ ]:
# Configure data loader
"""
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)
"""
dataloader = torch.utils.data.DataLoader(
    preset_dataset,
    batch_size=opt.batch_size,
    shuffle=True,
)

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
import csv

def save_sample(data,i,num=8):
    output = data.cpu().detach().numpy()
    #print(output[0])
    name = "./wgangp_final/"+str(i)+".csv"
    header_row = ["x"]*data_size
    with open(name, "w") as f:
        writer = csv.writer(f)
        writer.writerow(header_row)
        writer.writerows(output)

In [ ]:

def checkpoint_models(gen,desc,check):
    torch.save(gen.state_dict(), "./checkpoints/generator_"+str(check)+".pt")
    torch.save(desc.state_dict(), "./checkpoints/desc_"+str(check)+".pt")
    check = check+1

In [ ]:

batches_done = 0
for epoch in range(opt.n_epochs):
    for i, imgs in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

            if batches_done % opt.sample_interval == 0:
                print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
                )
                #save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
                save_sample(fake_imgs,batches_done)
                
            batches_done += opt.n_critic
        
        if epoch % 500 == 0:
            checkpoint_models(generator, discriminator,epoch)
